<a href="https://colab.research.google.com/github/scottieballs/weather_classifier/blob/master/weatherClassifierCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weather Image Classifier

This python notebook contains the implementation of a convolutional neural network (CNN)
for classifiying weather images into the following six classes:

1. Sunny
1. Cloudy
1. Rainy
1. Foggy
1. Other

The dataset used is the Image2Weather dataset published by 

Wei-Ta Chu, Xiang-You Zheng, and Ding-Shiuan Ding, "Camera as Weather Sensor: Estimating Weather Information from Single Images," Journal of Visual Communications and Image Representation, vol. 46, pp. 233-249, 2017.



In [ ]:
# Import modules
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import math
import PIL
import sys

from PIL import Image
sys.modules['Image'] = Image

print(tf.__version__)
tf.test.gpu_device_name()

# !pip install gputil
# import psutil
# import humanize
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

2.2.0


'/device:GPU:0'

In [ ]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

cwd_path = os.path.abspath(os.getcwd())
print(cwd_path)

zip_file = cwd_path +'/gdrive/My\ Drive/data/PreparedWeatherImages.zip'
if (os.path.isdir('/tmp/data') == False):
  os.mkdir('/tmp/data')
!cp $zip_file '/tmp/data'

# Unzip the dataset locally
local_zip = '/tmp/data' + '/PreparedWeatherImages.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
zip_ref.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content


In [ ]:
# Set the training and validation directories
train_dir = '/tmp/data/WeatherImages'
validation_dir = '/tmp/data/ValidationWeatherImages'

# Set plot_vars = True if you want to see pictures of the data 
plot_vars = False

In [ ]:
if plot_vars == True:
  # Plot some of the images in the data set
  %matplotlib inline

  nrows = 1
  ncols = 6

  fig = plt.gcf()
  fig.set_size_inches(ncols * 4, nrows * 4)

  image_list = []
  for d in os.listdir(train_dir):
      train_subdir = train_dir + '/' + d
      filename = os.listdir(train_subdir)[0]
      image_list.append(os.path.join(train_subdir, filename))

  for img_path in image_list:
      sp = plt.subplot(nrows, ncols, image_list.index(img_path)+1)
      sp.axis('Off')
      
      img = mpimg.imread(img_path)
      plt.imshow(img)

  plt.show()


In [ ]:
# Define callback class to stop training at 95 percent accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

checkpoint_path = cwd_path + '/gdrive/My Drive/data/training/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 198, 198, 32)      896       
_________________________________________________________________
batch_normalization_20 (Batc (None, 198, 198, 32)      128       
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 196, 196, 32)      9248      
_________________________________________________________________
batch_normalization_21 (Batc (None, 196, 196, 32)      128       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
batch_normalization_22 (Batc (None, 96, 96, 64)       

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0001),
             metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.05,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (200, 200),
    batch_size = 128,
    class_mode = 'categorical')

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical')

Found 147041 images belonging to 6 classes.
Found 36757 images belonging to 6 classes.


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = 400,
                    epochs = 10,
                    validation_data=validation_generator,
                    verbose = 1,
                    callbacks = [callbacks, cp_callback])

Epoch 1/10
400/400 [==============================] - ETA: 0s - loss: 0.9352 - accuracy: 0.6864
Epoch 00001: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
400/400 [==============================] - 937s 2s/step - loss: 0.9352 - accuracy: 0.6864 - val_loss: 1.0107 - val_accuracy: 0.5480
Epoch 2/10
400/400 [==============================] - ETA: 0s - loss: 0.7155 - accuracy: 0.7258
Epoch 00002: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
400/400 [==============================] - 938s 2s/step - loss: 0.7155 - accuracy: 0.7258 - val_loss: 0.7585 - val_accuracy: 0.6852
Epoch 3/10
400/400 [==============================] - ETA: 0s - loss: 0.6659 - accuracy: 0.7497
Epoch 00003: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
400/400 [==============================] - 935s 2s/step - loss: 0.6659 - accuracy: 0.7497 - val_loss: 0.7237 - val_accuracy: 0.7208
Epoch 4/10
400/400 [==============================] - ETA: 0s - loss: 0.6428 - accuracy:

In [ ]:

model.save(cwd_path + '/gdrive/My Drive/Colab Notebooks/WeatherClassifier_model_v4.h5') 